In [5]:
!python3 pip install -q tensorflow_datasets

python3: can't open file '/Users/cytalladen/Documents/datacamp/streamlit_apps/cifar10_img_class/pip': [Errno 2] No such file or directory


In [6]:
import tensorflow as tf
import tensorflow_datasets as tfds
# download the cifar10 dataset into the data directory
dataset, info = tfds.load('cifar10', with_info=True, as_supervised=True,data_dir='data')



/Users/cytalladen/opt/anaconda3/envs/tf/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-18 09:20:28.534744: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "FAILED_PRECONDITION: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...:   0%|                                  | 0/1 [00:00<?, ? url/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Size...:   0%|                                     | 0/162 [00:01<?, ? MiB/s]

Extraction completed...: 0 file [00:01, ? file/s]
Dl Size...:   1%|▏                            | 1/162 [00:02<05:44,  2.14s/ MiB]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Size...:   1%|▎                            | 2/162 [00:02<02:30,  1.06 MiB/s]

Dl Size...:   2%|▌                            | 3/162 [00:02<02:29,  1.06 MiB/s]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Size...:   2%|▋                            | 4/162 [00:02<01:02,  2.52 MiB/s]

Dl Size...:   3%|▉                            | 5/162 [00:02<01:02,  2.52 MiB/s]

Extraction completed...: 0 file [00:02, ? file/s]
Dl Size...:   4%|█                            | 6/162 [00:02<00:38,  4.05 MiB/s]

Dl Size...:   4%|█▎              

Dataset cifar10 downloaded and prepared to data/cifar10/3.0.2. Subsequent calls will reuse this data.


In [7]:
#split into train and test datasets
train_dataset, test_dataset = dataset['train'], dataset['test']


In [8]:

#create a function to normalize and resize the images
def normalize_and_resize(image, label):
    image=tf.cast(image, tf.float32)
    image=tf.divide(image, 255)
    image=tf.image.resize(image, [28,28])
    return image, label

#create a function to augment the images
def augment(image, label):
    image=tf.image.random_flip_left_right(image)
    image=tf.image.random_flip_up_down(image)
    image=tf.image.random_brightness(image, max_delta=0.5)
    image=tf.image.random_contrast(image, lower=0.2, upper=1.8)
    image=tf.image.random_hue(image, max_delta=0.2)
    image=tf.image.random_saturation(image, lower=0.2, upper=1.8)
    return image, label

In [9]:

#modify the train and test datasets using the function
train_dataset=train_dataset.map(normalize_and_resize).cache().map(augment).shuffle(1000).batch(64).repeat()
test_dataset=test_dataset.map(normalize_and_resize).cache().batch(64)



In [10]:

#create a basic CNN model using Tensorflow
basic_model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [11]:

basic_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#create an early stopping callback
early_stop=tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)

#create a lr plateau callback
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.001)



In [12]:

#fit the model with the data and the callbacks
history=basic_model.fit(train_dataset, epochs=32, steps_per_epoch=60000//64, validation_data=test_dataset, callbacks=[early_stop, reduce_lr],verbose=2)

Epoch 1/32
937/937 - 51s - loss: 1.8483 - accuracy: 0.3478 - val_loss: 1.4954 - val_accuracy: 0.4603 - lr: 0.0010 - 51s/epoch - 54ms/step
Epoch 2/32
937/937 - 44s - loss: 1.4831 - accuracy: 0.4716 - val_loss: 1.3680 - val_accuracy: 0.5168 - lr: 0.0010 - 44s/epoch - 47ms/step
Epoch 3/32
937/937 - 40s - loss: 1.3171 - accuracy: 0.5381 - val_loss: 1.1778 - val_accuracy: 0.5856 - lr: 0.0010 - 40s/epoch - 43ms/step
Epoch 4/32
937/937 - 46s - loss: 1.2181 - accuracy: 0.5734 - val_loss: 1.1352 - val_accuracy: 0.6003 - lr: 0.0010 - 46s/epoch - 49ms/step
Epoch 5/32
937/937 - 46s - loss: 1.1508 - accuracy: 0.5975 - val_loss: 1.1126 - val_accuracy: 0.6114 - lr: 0.0010 - 46s/epoch - 49ms/step
Epoch 6/32
937/937 - 43s - loss: 1.0998 - accuracy: 0.6143 - val_loss: 1.1085 - val_accuracy: 0.6117 - lr: 0.0010 - 43s/epoch - 46ms/step
Epoch 7/32
937/937 - 45s - loss: 1.0568 - accuracy: 0.6302 - val_loss: 1.0976 - val_accuracy: 0.6222 - lr: 0.0010 - 45s/epoch - 48ms/step
Epoch 8/32
937/937 - 45s - loss: 1

In [14]:

# https://www.tensorflow.org/api_docs/python/tf/keras/saving/save_model
# save to hdf5 file
# Note that model.save() is an alias for tf.keras.saving.save_model().
# save to keras savedfile

# Calling `save('my_model.keras')` creates a zip archive `my_model.keras`.
basic_model.save("basic_model.keras")



In [ ]:

# Upon loading, pass a dict containing the custom objects used in the
# `custom_objects` argument of `keras.models.load_model()`.
# reloaded_model = keras.models.load_model(
#     "basic_model.keras",
#     custom_objects={"CustomLayer": CustomLayer, "custom_fn": custom_fn},
# )

